In [ ]:
# Instalowanie potrzebnych bibliotek
! pip install woocommerce
! pip install google-cloud-bigquery
! pip install simplejson

In [2]:
# Import potrzebnych bibliotek
from google.cloud import bigquery
from woocommerce import API
import json
from datetime import datetime, timedelta
import re
import hashlib

In [ ]:
# Autoryzacja
wcapi = API(
    url="https://przykladowastrona.com/",
    consumer_key="TWOJ-KLUCZ-KONUSMENTA",
    consumer_secret="TWOJ-SERETNY-KLUCZ",
    version="wc/v3",
    timeout=30  # Zwiększenie czasu oczekiwania na odpowiedź (30 sekund)
)

In [17]:
# Ustawienia dla pobrania zamówień
start_date = "2024-01-01T00:00:01"  # Początek roku w formacie ISO 8601
end_date = "2025-05-31T23:59:59"  # Koniec okresu w formacie ISO 8601 (poprawiona data)

# Prawidłowy format dat dla wczorajszego dnia
yesterday_start = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%dT00:00:01")  # Wczoraj 00:00:01
yesterday_end = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%dT23:59:59")    # Wczoraj 23:59:59

per_page = 20  # Maksymalna liczba wyników na stronę
page = 1  # Startowa strona

all_orders = []  # Lista do przechowywania wszystkich zamówień

In [ ]:
# Pobieranie zamówień o statusie "completed" z WooCommerce
while True:
    response = wcapi.get(f"orders?status=completed,processing&after={yesterday_start}&before={yesterday_end}&per_page={per_page}&page={page}")

    if response.status_code == 200:
        orders = response.json()
        if not orders:
            # Przerwij pętlę, gdy nie ma więcej zamówień
            break
        all_orders.extend(orders)
        page += 1
    else:
        print("Błąd:", response.status_code, response.text)
        break

# Po zakończeniu pobierania danych
print("Łączna liczba zamówień:", len(all_orders))
print("Przykładowe zamówienie:", all_orders[0] if all_orders else "Brak zamówień")

In [ ]:
# # (Alternatywa z filtrowaniem) Pobieranie zamówień o statusie "completed" z WooCommerce
# while True:
#     response = wcapi.get(f"orders?status=completed,processing&after={start_date}&before={end_date}&per_page={per_page}&page={page}")

#     if response.status_code == 200:
#         orders = response.json()
#         if not orders:
#             # Przerwij pętlę, gdy nie ma więcej zamówień
#             break

#         # Filtrujemy zamówienia, aby pobierać tylko te z Polski
#         filtered_orders = [order for order in orders if order.get('billing', {}).get('country') == 'PL']

#         if filtered_orders:
#             all_orders.extend(filtered_orders)

#         page += 1
#     else:
#         print("Błąd:", response.status_code, response.text)
#         break

In [20]:
# Funkcje do przygotowania importu customer list dla Google Ads

# Funkcja do formatowania emaila
def format_email(email):
    if email:
        return email.strip().lower()  # Usuwamy białe znaki i zamieniamy na małe litery
    return None

# Funkcja do formatowania numeru telefonu
def format_phone(phone):
    if phone:
        # Usuwamy wszystkie niepotrzebne znaki (spacje, myślniki, nawiasy) i zostawiamy tylko cyfry i '+'
        phone = re.sub(r'[^+\d]', '', phone)

        # Sprawdzamy, czy telefon zaczyna się od '+48', jeśli nie, dodajemy '+48' na początku
        if not phone.startswith('+48'):
            phone = '+48' + phone.lstrip('0')  # Usuwamy ewentualne '0' na początku numeru, jeśli jest

        return phone
    return None

# Funkcja do zaszyfrowania danych za pomocą SHA256
def hash_sha256(value):
    if value:
        return hashlib.sha256(value.encode('utf-8')).hexdigest()
    return None

In [22]:
# Konwersja danych zamówień do listy słowników
order_data = []
for order in all_orders:
    total_value = order.get("total")
    if total_value is not None:
        total_value = float(order.get("total", 0))
        shipping_total = float(order.get("shipping_total", 0))
        total_tax = float(order.get("total_tax", 0))

    # Zaszyfrowanie emaila i telefonu
    billing_email = hash_sha256(format_email(order.get("billing", {}).get("email")))
    billing_phone = hash_sha256(format_phone(order.get("billing", {}).get("phone")))

    order_data.append({
        "id": order.get("id"),
        "total_netto": round(total_value - (shipping_total + total_tax), 2),
        "date_created": order.get("date_created"),
        "customer_id": order.get("customer_id"),
        "email": billing_email,
        "phone": billing_phone,
        # Ustawienie order_items jako lista RECORD
        "order_items": [
            {
                "product_id": item.get("product_id"),
                "product_sku": item.get("sku"),
                "product_name": item.get("name"),
                "quantity": item.get("quantity"),
                "price": round(float(item.get("price", 0)), 2) if item.get("price") else 0,
                "subtotal": item.get("subtotal"),
            } for item in order.get("line_items", [])
        ]
    })

In [ ]:
# Inicjalizacja klienta BigQuery
client = bigquery.Client()

# Konfiguracja schematu tabeli BigQuery
table_id = "TWOJA-SCIEZKA-DO-TABELI-BIGQUERY"
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("total_netto", "FLOAT"),
        bigquery.SchemaField("date_created", "TIMESTAMP"),
        bigquery.SchemaField("customer_id", "INTEGER"),
        bigquery.SchemaField("email", "STRING"),
        bigquery.SchemaField("phone", "STRING"),
        bigquery.SchemaField(
            "order_items",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("product_id", "INTEGER"),
                bigquery.SchemaField("product_sku", "STRING"),
                bigquery.SchemaField("product_name", "STRING"),
                bigquery.SchemaField("quantity", "INTEGER"),
                bigquery.SchemaField("price", "FLOAT"),
                bigquery.SchemaField("subtotal", "FLOAT"),
            ]
        ),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    # Add time partitioning configuration
    time_partitioning=bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="date_created",
        expiration_ms=None,
    )
)

# Wczytanie danych JSON do BigQuery
job = client.load_table_from_json(order_data, table_id, job_config=job_config)
job.result()  # Czekanie na zakończenie przesyłania
print("Dane JSON przesłane do BigQuery wraz z kategoryzacją produktów!")